# Access Control - Assigning a holder

This notebook will demonstrate to pass the ownership of a resource to another originator. 

- Create a new resource without own *accessControlPolicyIDs* attribute
- Assign the ownership of that resource to someone else


## Intitialization
<div class="alert alert-block alert-info" style="border-radius: 10px;">
This notebook relies on the resources created in the exercise's previous notebooks.
<br/>
    
The following section imports and initializes necessary functions and configurations. 
It must be executed once for each notebook before running any of cells.
<br/>
    
<b>Also make sure that the CSE is running!</b>
</div>

In [ ]:
%run ../src/init.py

## Bob adds a \<container> resource

This time Bob will create a new \<container> resource under his \<AE> resource:

In [ ]:
CREATE (                                           # CREATE request
    target                  = f'{cseRN}/aeBob',    # Create the <cnt> under aeBob
    originator              = 'CBob',              # Request's originator (mandatory)
    requestIdentifier       = '123',               # Request identifier (mandatory)
    releaseVersionIndicator = '3',                 # Request's release version indicator (mandatory)
    resourceType            = tyContainer,         # The request creates an <container> resource

    # Request Content
    content = {
        'm2m:cnt': {
            'rn': 'bobsContainer'
        }
    }
)

## Bob tries to retrieve the \<container> resource

Bob tries to retrieve the newly created \<container> resource. This should succeed:

In [ ]:
RETRIEVE (                                                     # RETRIEVE request
    target                  = f'{cseRN}/aeBob/bobsContainer',  # Target resource is the <cnt> under Bob's <AE>
    originator              = 'CBob',                          # Request's originator is Bob (mandatory)
    requestIdentifier       = '123',                           # Request identifier (mandatory)
    releaseVersionIndicator = '3'                              # Request's release version indicator (mandatory)
)

As expected this works.

## Alice tries to retrieve Bob's \<container> resource

Now, Alice will try to retrieve Bob's \<container> resource. This should fail, because we have no extra access control policies created for this \<container>.

In [ ]:
RETRIEVE (                                                     # RETRIEVE request
    target                  = f'{cseRN}/aeBob/bobsContainer',  # Target resource is the <cnt> under Bob's <AE>
    originator              = 'CAlice',                        # Request's originator is Alice (mandatory)
    requestIdentifier       = '123',                           # Request identifier (mandatory)
    releaseVersionIndicator = '3'                              # Request's release version indicator (mandatory)
)

As expected this **fails** as Alice has no permissions and so the request fails.

## Bob passes on the ownership of the \<container> resource

With the following request Bob gives ownership of the \<container> resource to Alice:

In [ ]:
UPDATE (                                                      # UPDATE request
    target                  = f'{cseRN}/aeBob/bobsContainer', # Update the <cnt>
    originator              = 'CBob',                         # Request's originator (mandatory)
    requestIdentifier       = '123',                          # Request identifier (mandatory)
    releaseVersionIndicator = '3',                            # Request's release version indicator (mandatory)

    # Request Content
    content = {
        'm2m:cnt': {
            'hld': 'CAlice'
        }
    }
)

## Bob tries again to retrieve the \<container> resource

Bob tries again to retrieve the \<container> resource. But this time the request should fail:

In [ ]:
RETRIEVE (                                                     # RETRIEVE request
    target                  = f'{cseRN}/aeBob/bobsContainer',  # Target resource is the <container> under Bob's <AE>
    originator              = 'CBob',                          # Request's originator is Bob (mandatory)
    requestIdentifier       = '123',                           # Request identifier (mandatory)
    releaseVersionIndicator = '3'                              # Request's release version indicator (mandatory)
)

This **failed**. What happend here?

## Alice tries again to retrieve the \<container> resource

Now Alice will also try again to retrieve the \<container> resource. Last time this didn't succeed. How about now?

In [ ]:
RETRIEVE (                                                     # RETRIEVE request
    target                  = f'{cseRN}/aeBob/bobsContainer',  # Target resource is the <cnt> under Bob's <AE>
    originator              = 'CAlice',                        # Request's originator is Alice (mandatory)
    requestIdentifier       = '123',                           # Request identifier (mandatory)
    releaseVersionIndicator = '3'                              # Request's release version indicator (mandatory)
)

Yes, this worked!

With setting the *holder* attribute of a resource the original creator can assign the resource to another originator. This is quiet convenient in cases where one application acts as a factory and creates resources for one or more other AEs.

<div class="alert alert-block alert-success" style="border-radius: 10px">
    This access control procedure is used when there is no <i>accessControlPolicyIDs</i> attribute is set for a resource. It  takes precedense before the access control check for the creator of a resource (ie. the default policy when no other access control policy is assigned).
<br/> 
The current owner of a resource can give up ownership by removing the *holder* attribute in an UPDATE request, or assigning even another originator to this attribute and thereby passing on the ownership again.    
</div>



&nbsp;